In [1]:
### 순서가 없는 범주 특성 인코딩하기
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

# 특성
feature=np.array([['서울'],['경기'],['인천'],['경기'],['인천']])

# 원-핫 인코더
one_hot=LabelBinarizer()

# 특성을 원-핫 인코딩
one_hot.fit_transform(feature)

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]])

In [2]:
# 특성의 클래스를 확인
one_hot.classes_

array(['경기', '서울', '인천'], dtype='<U2')

In [4]:
# 원-핫 인코딩된 데이터를 복원
one_hot.inverse_transform(one_hot.transform(feature))

array(['서울', '경기', '인천', '경기', '인천'], dtype='<U2')

In [7]:
### 순서가 있는 범주형 특성 인코딩하기
import pandas as pd

# 특성
df=pd.DataFrame({"Score": ['Low','Low','Medium','Medium','High']})

# 매핑 딕셔너리
scale_mapper={"Low":1, "Medium":2, "High":3}

# 특성을 정수로 변환
df["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

In [8]:
df=pd.DataFrame({"Score":['낮다','낮다','보통이다','보통이다','높다','약간 보통이다']})

# 클래스의 간격이 동일하지 않음
scale_mapper={"낮다":1, "보통이다":2, "약간 보통이다":3, "높다":4}
# "보통이다":2 에 "높다":3 을 주고 "약간 보통이다"에 실수를 줄 수가 없는 걸까...?

df['Score'].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    4
5    3
Name: Score, dtype: int64

In [9]:
# 클래스의 간격이 동일하도록 설정해야 함
scale_mapper={"낮다":1, "보통이다":2, "약간 보통이다":2.5, "높다":3}
# 뭐야.. 되네

df['Score'].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
5    2.5
Name: Score, dtype: float64

In [11]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

features=np.array([["Low",3],["Medium",2],["High",1]])

# 클래스 범주와 숫자가 일치하도록 변환함
ordinal_encoder=OrdinalEncoder()
ordinal_encoder.fit_transform(features)

array([[1., 2.],
       [2., 1.],
       [0., 0.]])

In [12]:
# 문자열은 문자 코드 순으로 정렬
# 숫자도 문자 코드 순으로 정렬
# 같은 문자 코드끼리 조합
# 인덱스 / 라벨
#   0       1    High
#   1       2    Low
#   2       3    Medium
ordinal_encoder.categories_

[array(['High', 'Low', 'Medium'], dtype='<U6'),
 array(['1', '2', '3'], dtype='<U6')]

In [4]:
### 누락된 클래스 값 대체
# 범주형 변수의 결측값-1. KNN 분류 모형으로 분류하여 채우는 방법
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# 범주형 특성을 가진 특성 행렬
X=np.array([[0,2.10,1.45],[1,1.18,1.33],[0,1.22,1.27],[1,-0.21,-1.19]])

# 누락된 값이 있는 데이터
X_with_nan=np.array([[np.nan,0.87,1.31],[np.nan,-0.67,-0.22]])

# KNN 모형 훈련
clf=KNeighborsClassifier(3)
trained_model=clf.fit(X[:,1:], X[:,0])

# 누락된 값의 클래스를 예측
imputed_values=trained_model.predict(X_with_nan[:,1:])

# 예측된 클래스와 원본 특성을 열로 병합
X_with_imputed=np.hstack((imputed_values.reshape(-1,1), X_with_nan[:,1:]))

# 두 특성 행렬을 연결
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

In [5]:
# 범주형 변수의 결측값-1. SimpleImputer를 사용하는 방법
from sklearn.impute import SimpleImputer

# 두 개의 특성 행렬을 병합
X_complete=np.vstack((X_with_nan, X))

imputer=SimpleImputer(strategy='most_frequent')
imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])